# An Introduction to Neural Networks

Neural networks are one of the hottest areas of machine learning. Neural network models attract the most press because of their relationship to the human brain enabling the anthropomorphism of artificial intelligence further. Neural networks aim to emulate the neurons of the brain with perceptrons linking them in networks.

Humans still maintain the edge in the comparison of human and computer intelligence and a look at the wetware specs of the brain and the hardware specs of computers gives us an idea as to why.

### Human Brain
* Neuron Switching Time:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$0.001\text{s}$ ($1\text{ms}$)
* Number of Neurons:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$10^{10}$
* Connections per Neuron:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$10^4-10^5$
* Scene Recognition Time:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$~0.1\ Second$

### Computer
* Transistor Switching Time:&nbsp;&nbsp;&nbsp;&nbsp;$~333\ picoseonds$ (3GHz Processor)
* Transistors per Chip:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$8 - 20\ billion$
* Connections per Transisor:&nbsp;&nbsp;&nbsp;$~10$
* Scene Recognition Time:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;No general scene recognition progamme yet exists (The _Inception_ model is getting pretty close).

We may see that computer hardware wins the comparison hands down in switching time and is starting to win in terms of number of transistors/neurons but the human brain seems way more powerful as a general purpose computing unit at the moment. The power of this comes through the connections per neuron. It is currently not forseen that computing hardware of the current paradigm will ever overtake the brain in this respect. The connections between neurons (albeit slow, exhchanging information via synapses) are where the brain learns and maintains its computational superiority.

Furthermore the brain manages this power with power consumption on the order of 100 Watts whereas the computers used to train and build the leading neural network models consume far far more energy. The brain manages to achieve massive parallelism through the connections of neurons from one to another and thereby achieves remarkable efficiency and understanding.

## The Neuron and the Perceptron
The software equivalent of the neuron is the perceptron. A single neuron is able to encode a single binary threshold which could be sufficent for a simple binary classification model. The neuron either fires or doesn't. In the case that the incoming stimulus is sufficient to cause the neuron to fire an action potential is sent down the axon and down the dendrites which then passes the stimulus on to the neurons connected to firing neuron. This threshold will differ from neuron to neuron as will the number and paths of the connections between neurons.

![Neuron Diagram](./img/neuron.png "A Neuron")
<center><span style="color:gray">*Source: Wikimedia Commons*</span></center>

The perceptron tries to emulate this mathematically by taking in a series of inputs summing them (scaled by some weights) and comparing them to a threshold.

![Perceptron Diagram](./img/perceptron.png "The Perceptron")
<center><span style="color:gray">*Source: Wikimedia Commons (Edited)*</span></center>

In the diagram the threshold is given by the constant $\theta$ which is also referred to as the *bias*. The inputs are given by the *n*-vector ***x***. The *x* values are then multiplied by their respective weights and summed before the result is passed through the step function which then leads to a binary output ($1$ '*fire*' or $0$ '*do not fire*'). This step function is known as the *activation function* since it determines the rule as to whether or not the perception '*fires*'.

Mathematically the perceptron (in the case of the purest neuron emulation) undertakes the following calculation:

$$output = \begin{cases} 1 &\mbox{if    } \left(w_0 + \sum_{i=1}^n w_ix_i\right)>0 \\ 
0 & \mbox{otherwise} \end{cases}$$

The output of the perceptron can then be passed forwards to other perceptrons akin to the action potential passing from the dendrites of one neuron to the next neuron through the chemical mediation of a synapse (which is very slow by computing standards). In this way perceptrons are built up to generate a neural network.

Taking the single perceptron as an example we may see that it characterises a hyperplane which implements a binary classification. The gradient of the hyperplane is determined by the weights vector $\boldsymbol{w}$ and its position is given by the bias $w_0 = \theta$.